<a href="https://colab.research.google.com/github/lionelfragniere/SSD_EORE/blob/master/SSD_EORE_DRAFT_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Mount Google Drive
from google.colab import drive; drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Install geospatial stack, perform necessary imports
!pip -q install geopandas pyogrio shapely rtree pyproj rasterio rioxarray xarray pyarrow statsmodels rasterstats

import os, json, hashlib, textwrap, re
from datetime import date
import pandas as pd
import geopandas as gpd
import numpy as np
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import rasterio
from rasterio.features import rasterize
from rasterio.enums import Resampling
from rasterstats import zonal_stats

# Define base parameters
BASE = "/content/drive/MyDrive/SSD_EORE"
SNAPSHOT = "2025-10-08"  # update as needed

# Folders
PATHS = {
    "inputs": f"{BASE}/data/inputs/{SNAPSHOT}",
    "processed": f"{BASE}/data/processed",
    "outputs": f"{BASE}/data/outputs",
    "config": f"{BASE}/config",
    "boundaries": f"{BASE}/data/inputs/{SNAPSHOT}/boundaries"
}
for p in PATHS.values():
    os.makedirs(p, exist_ok=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 78.2 MB/s eta 0:00:00


In [10]:
# Helper for manifest and checksums
def sha256sum(path, block_size=65536):
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(block_size), b""):
            h.update(chunk)
    return h.hexdigest()

manifest_rows = []

def register_file(dataset_id, title, provider, path, boundary_version=None, license_str="open", processing="raw"):
    row = {
        "dataset_id": dataset_id,
        "title": title,
        "provider": provider,
        "path": path,
        "snapshot_date": str(date.today()),
        "license": license_str,
        "processing": processing,
        "boundary_version": boundary_version if boundary_version else ""
    }
    row["sha256"] = sha256sum(path) if os.path.exists(path) else ""
    manifest_rows.append(row)


In [3]:
# Population per year and per admin3, processed from tiffs
# Source: WorldPop
# https://hub.worldpop.org/geodata/listing?id=135
POP_DIR = f"{BASE}/data/inputs/{SNAPSHOT}/population"
BND_FILE = f"{BASE}/data/inputs/{SNAPSHOT}/boundaries/ssd_admin0_3.gpkg"
OUT_CSV = f"{BASE}/data/processed/pop_admin3_worldpop.csv"
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

adm = gpd.read_file(BND_FILE)
rename_map = {
    'ADM1_PCODE':'adm1_pcode','ADM1_EN':'adm1_name',
    'ADM2_PCODE':'adm2_pcode','ADM2_EN':'adm2_name',
    'ADM3_PCODE':'adm3_pcode','ADM3_EN':'adm3_name',
    'ISO3':'country_iso3'
}
for old,new in rename_map.items():
    if old in adm.columns and new not in adm.columns:
        adm = adm.rename(columns={old:new})

need = ['adm3_pcode','adm3_name','adm2_pcode','adm2_name','adm1_pcode','adm1_name','geometry']
missing = [c for c in need if c not in adm.columns]
if missing:
    raise ValueError(f"Boundaries missing fields: {missing}")

# Fix invalid geometries
adm = adm.set_crs(4326, allow_override=True)
adm["geometry"] = adm.buffer(0)
adm = adm[~adm.geometry.is_empty]

def zonal_sum_fast(tif_path, adm3_gdf, pcode_field="adm3_pcode"):
    year_match = re.search(r'(\d{4})', os.path.basename(tif_path))
    year = int(year_match.group(1)) if year_match else None

    with rasterio.open(tif_path) as src:
        # Reproject admin3 to raster CRS
        adm_r = adm3_gdf.to_crs(src.crs)

        # Map each adm3 to a small integer label
        codes = adm_r[pcode_field].astype("category")
        adm_r["zone_id"] = codes.cat.codes.astype(np.int32) + 1  # 0 reserved for background
        zone_ids = adm_r["zone_id"].to_numpy()
        zone_idx_to_pcode = dict(zip(zone_ids, codes.astype(str)))

        # Rasterise zone labels at raster resolution
        shapes = list(zip(adm_r.geometry, zone_ids))
        zone_raster = rasterize(
            shapes=shapes,
            out_shape=(src.height, src.width),
            transform=src.transform,
            fill=0,
            dtype="int32"
        )

        # Read population data as float32
        pop = src.read(1, out_dtype="float32", masked=True)
        # Mask where zone is 0 or pop is masked
        mask = (zone_raster == 0) | np.ma.getmaskarray(pop)
        zone = np.where(mask, 0, zone_raster).ravel()
        vals = np.where(mask, 0.0, pop.filled(0)).ravel()

        # Sum by zone id
        sums = np.bincount(zone, weights=vals, minlength=zone_raster.max()+1)
        # Build DataFrame for non‑zero zones
        rows = []
        for zid in np.unique(zone_raster):
            if zid == 0:
                continue
            pcode = zone_idx_to_pcode.get(zid)
            rows.append((pcode, year, float(sums[zid])))

        df = pd.DataFrame(rows, columns=["adm3_pcode","year","pop_total"])
        return df

# Process all WorldPop files found in POP_DIR
tif_files = [os.path.join(POP_DIR, f) for f in sorted(os.listdir(POP_DIR)) if f.lower().endswith(".tif")]
pop_list = [zonal_sum_fast(t, adm) for t in tif_files]
pop_admin3 = pd.concat(pop_list, ignore_index=True)

# Round to integers for reporting clarity
pop_admin3["pop_total"] = pop_admin3["pop_total"].round().astype("int64")
pop_admin3.to_csv(OUT_CSV, index=False)
# Check for missing or zero denominators
missing = pop_admin3[pop_admin3["pop_total"] <= 0]["adm3_pcode"].nunique()
print(f"Admin3 units with zero or missing population: {missing}")

# Spot‑check totals by year
totals = pop_admin3.groupby("year")["pop_total"].sum().reset_index()
print(totals.head())

# Merge back names for readability
pop_admin3_named = pop_admin3.merge(adm[["adm3_pcode","adm3_name","adm2_name","adm1_name"]], on="adm3_pcode", how="left")
pop_admin3_named.head()


Admin3 units with zero or missing population: 1
   year  pop_total
0  2018    9926665
1  2019   10251356
2  2020   10525162
3  2021   10799304
4  2022   10858417


,adm3_pcode,year,pop_total,adm3_name,adm2_name,adm1_name
0,SS000101,2018,30,Abyei Region,Abyei Region,Abyei Region
1,SS010101,2018,893,Bungu,Juba,Central Equatoria
2,SS010102,2018,6783,Dolo,Juba,Central Equatoria
3,SS010103,2018,1277,Ganji,Juba,Central Equatoria
4,SS010104,2018,30017,Gondokoro,Juba,Central Equatoria


In [11]:
# Admin3 Boudaries processing
# Source = HDX
BOUNDARY_FILE = f"{PATHS['inputs']}/boundaries/ssd_admin0_3.gpkg"
boundary_version = "COD-AB v20230901"

adm = gpd.read_file(BOUNDARY_FILE, layer=None)  # auto detect layer if single-layer gpkg
# Make sure fields exist; rename if needed
rename_map = {
    'ADM1_PCODE':'adm1_pcode','ADM1_EN':'adm1_name',
    'ADM2_PCODE':'adm2_pcode','ADM2_EN':'adm2_name',
    'ADM3_PCODE':'adm3_pcode','ADM3_EN':'adm3_name',
    'ADM0_PCODE':'country_iso3'
}
for old,new in rename_map.items():
    if old in adm.columns and new not in adm.columns:
        adm = adm.rename(columns={old:new})

needed = ['country_iso3','adm1_pcode','adm1_name','adm2_pcode','adm2_name','adm3_pcode','adm3_name','geometry']
missing = [c for c in needed if c not in adm.columns]
if missing:
    raise ValueError(f"Missing fields in boundaries: {missing}")

adm["boundary_version"] = boundary_version
adm["snapshot_date"] = str(date.today())
adm = adm.set_crs(4326)

register_file("boundaries_admin3", "South Sudan admin 0–3", "OCHA COD via HDX", BOUNDARY_FILE, boundary_version)
adm.head(5)


,Shape_Leng,Shape_Area,adm3_name,adm3_pcode,adm2_name,adm2_pcode,adm1_name,adm1_pcode,ADM0_EN,country_iso3,date,validOn,validTo,AREA_SQKM,geometry,boundary_version,snapshot_date
0,3.676016,0.864956,Abyei Region,SS000101,Abyei Region,SS0001,Abyei Region,SS00,South Sudan,SS,2023-08-29,2023-08-29,NaT,10495.845837,"POLYGON ((28.03855 9.3439, 28.03606 9.34387, 2...",COD-AB v20230901,2025-10-09
1,1.096062,0.054905,Bungu,SS010101,Juba,SS0101,Central Equatoria,SS01,South Sudan,SS,2023-08-29,2023-08-29,NaT,673.619210,"POLYGON ((31.20592 4.81821, 31.21309 4.81718, ...",COD-AB v20230901,2025-10-09
2,1.791721,0.149619,Dolo,SS010102,Juba,SS0101,Central Equatoria,SS01,South Sudan,SS,2023-08-29,2023-08-29,NaT,1835.038343,"POLYGON ((31.26447 5.13648, 31.28455 5.12549, ...",COD-AB v20230901,2025-10-09
3,0.762490,0.029053,Ganji,SS010103,Juba,SS0101,Central Equatoria,SS01,South Sudan,SS,2023-08-29,2023-08-29,NaT,356.529440,"POLYGON ((31.21002 4.62885, 31.2182 4.62271, 3...",COD-AB v20230901,2025-10-09
4,1.232994,0.051823,Gondokoro,SS010104,Juba,SS0101,Central Equatoria,SS01,South Sudan,SS,2023-08-29,2023-08-29,NaT,635.698071,"POLYGON ((31.7016 5.00761, 31.7164 4.99368, 31...",COD-AB v20230901,2025-10-09


In [6]:
# ACLED file processing
# Source = ACLED, filtered for Explosive ordnance-related incidents only
acled_files = [os.path.join(PATHS["inputs"], "acled", f) for f in os.listdir(f"{PATHS['inputs']}/acled") if f.endswith(".csv")]
df_acled = pd.concat((pd.read_csv(f) for f in acled_files), ignore_index=True)

# Parse dates
df_acled["event_date"] = pd.to_datetime(df_acled["event_date"],
               format='%d/%m/%Y %H:%M:%S')
# Convert to GeoDataFrame and spatially join to admin3
acled_gdf = gpd.GeoDataFrame(df_acled, geometry=gpd.points_from_xy(df_acled.longitude, df_acled.latitude), crs=4326)
acled_join = gpd.sjoin(acled_gdf, adm[["adm3_pcode","geometry"]], how="left", predicate="within")
acled_join["year"] = acled_join["event_date"].dt.year
acled_join["month"] = acled_join["event_date"].dt.month

acled_m = (acled_join
           .groupby(["adm3_pcode","year","month"], as_index=False)
           .agg(acled_events=("event_date","count"),
                acled_fatalities=("fatalities","sum")))
register_file("acled_events", "ACLED events monthly admin3", "ACLED", acled_files[0], boundary_version)
acled_m.head(5)


,adm3_pcode,year,month,acled_events,acled_fatalities
0,SS010105,2016,7,2,1
1,SS010106,2024,11,1,0
2,SS010107,2022,3,1,0
3,SS010108,2020,4,1,0
4,SS010108,2020,8,1,0


In [12]:
# Seasonal Calendar Processing
# Source
BND_FILE = f"{BASE}/data/inputs/{SNAPSHOT}/boundaries/ssd_admin0_3.gpkg"
OUT_DIR = f"{BASE}/data/inputs/{SNAPSHOT}/seasonal"
os.makedirs(OUT_DIR, exist_ok=True)

adm = gpd.read_file(BND_FILE)
# Standardise names if needed
rename_map = {
    'ADM1_PCODE':'adm1_pcode','ADM1_EN':'adm1_name',
    'ADM2_PCODE':'adm2_pcode','ADM2_EN':'adm2_name',
    'ADM3_PCODE':'adm3_pcode','ADM3_EN':'adm3_name',
    'ISO3':'country_iso3'
}
for old,new in rename_map.items():
    if old in adm.columns and new not in adm.columns:
        adm = adm.rename(columns={old:new})

adm1 = adm[["adm1_pcode","adm1_name"]].drop_duplicates()

# Identify the three Equatoria states for a bimodal pattern
equatoria_like = {"Central Equatoria", "Eastern Equatoria", "Western Equatoria"}
adm1["zone_type"] = adm1["adm1_name"].apply(lambda x: "Greenbelt_Hills" if x in equatoria_like else "Other")

years = list(range(2018, 2026))
months = list(range(1, 13))

# Unimodal pattern for 'Other' (national default)
season_map_other = {
    1: "Dry", 2: "Dry", 3: "Dry",
    4: "Pre-planting", 5: "Pre-planting",
    6: "Rainy", 7: "Rainy", 8: "Rainy", 9: "Rainy",
    10: "Post-rainy", 11: "Dry", 12: "Dry"
}
lean_other = {6, 7, 8}
harvest_other = {10, 11, 12, 1}
flood_other = {8, 9, 10}

# Bimodal pattern for 'Greenbelt_Hills' (Equatorias)
season_map_greenbelt = {
    1: "Dry", 2: "Dry",
    3: "Pre-planting",
    4: "Rainy_1", 5: "Rainy_1", 6: "Rainy_1",
    7: "Harvest_1",
    8: "Pre-planting_2",
    9: "Rainy_2", 10: "Rainy_2", 11: "Rainy_2",
    12: "Harvest_2"
}
# For binary flags used in analysis we still mark "Rainy" periods as rainy
def season_flag_canonical(zone_type, m):
    if zone_type == "Greenbelt_Hills":
        if season_map_greenbelt[m].startswith("Rainy"):
            return "Rainy"
        if "Pre-planting" in season_map_greenbelt[m]:
            return "Pre-planting"
        if "Harvest" in season_map_greenbelt[m]:
            return "Post-rainy"
        return "Dry"
    else:
        return season_map_other[m]

lean_greenbelt = {6, 7}                 # shorter lean in some bimodal areas
harvest2_greenbelt = {11, 12, 1}        # second harvest window
harvest1_greenbelt = {7, 8}             # first harvest window
flood_greenbelt = {8, 9, 10}            # seasonal flooding still likely in late rainy months

rows = []
for y in years:
    for m in months:
        for _, row in adm1.iterrows():
            zone = row["zone_type"]
            season_canon = season_flag_canonical(zone, m)
            rows.append({
                "adm1_pcode": row["adm1_pcode"],
                "adm1_name": row["adm1_name"],
                "zone_type": zone,
                "year": y,
                "month": m,
                "season_flag": season_canon,
                "lean_season": 1 if ((m in lean_greenbelt) if zone=="Greenbelt_Hills" else (m in lean_other)) else 0,
                "seasonal_flooding": 1 if ((m in flood_greenbelt) if zone=="Greenbelt_Hills" else (m in flood_other)) else 0,
                "harvest_main": 1 if ((m in harvest2_greenbelt) if zone=="Greenbelt_Hills" else (m in harvest_other)) else 0
            })

season_by_adm1 = pd.DataFrame(rows)
season_by_adm1.to_csv(f"{OUT_DIR}/seasonal_calendar_by_adm1.csv", index=False)
season_by_adm1.head()

SEASON_FILE = f"{PATHS['inputs']}/seasonal/seasonal_calendar_by_adm1.csv"
# Expect columns: year, month, season_flag (and optionally adm1_pcode)
season = pd.read_csv(SEASON_FILE)
assert set(["year","month","season_flag"]).issubset(season.columns)
register_file("seasonal_calendar", "Seasonal calendar", "FEWS NET or programmed rules", SEASON_FILE)
